## Let's use hypothesis testing to see if data have the same mean
Recall, p-value is the probability of observing your data (or more extrem) given that the null hypothesis is true.

In our case:
**Null hypothesis** is that means are equal
**Alternative hyp.** is that they differ

If p-value < 5%, it means that it is quite unlikely to oberve the provided data under the assumption that the null hypothesis is true. Therefore, we **can** reject it in favor of the alternative hypothesis.


In [ ]:
# get filenames


production_file = "/home/durkota/edna/home/durkota/Experiment-13/omnia2/147.32.83.175.binetflow"
hp_file = "/home/durkota/edna/home/durkota/Experiment-13/omnia1/147.32.83.179.binetflow"
port = '80'


In [ ]:
#!/usr/bin/env python3.6 -u
# Author: Karel Durkota
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import flowExtractor 
import datetime
import datetime
import time

In [ ]:
from datetime import datetime

datetime_object = datetime.strptime('2018/02/13 18:07:08.713635', '%Y/%m/%d %X.%f')

In [ ]:
# get sequence of unique # of IP per hour 
def getHour(string):
    return string.split(" ")[1].split(":")[0]

def getDateAndHour(string):
    d = datetime.strptime(string, '%Y/%m/%d %X.%f')
    d = d.replace(microsecond=0,second=0,minute=0)
    return d

In [ ]:
def generateIntermediateHours(start, end):
    delta = timedelta(seconds=60*60)         # timedelta

    l = []
    
    d = start
    while d <= end:
#         print(d)
        d = d + delta
        l.append(d)
    return l
        

In [ ]:
def getSequenceFromData(data):
    first_date = getDateAndHour(data[1,1])
    last_date = getDateAndHour(data[data.shape[0]-1,1])
    dates = generateIntermediateHours(first_date, last_date)
    
    ser = pd.Series(data[1:,0]).groupby(list(map(getDateAndHour, data[1:,1]))).count()
    
    seq = []
    for d in dates:
        if d in ser.index:
            seq.append(ser.get(d))
        else:
            seq.append(0)
    return seq

In [ ]:
col = dict()

def getFullSequence(filename, port):
    global col
    seq = []
#    for filename in filenames:
    data = np.array(flowExtractor.extract(['-p', port, '-f', filename]))

    header = data[0,:]
    col = dict([(header[i], i) for i in range(len(header))])

    seq.append(getSequenceFromData(data[:,[col['SrcAddr'], col['StartTime']]]))
    return [y for x in seq for y in x]



In [ ]:
              
              
# getting full sequences
seq1 = getFullSequence(production_file, port)
seq2 = getFullSequence(hp_file, port)

# run hypothesis test
# Hypothesis testing


print("Mean1 = {0}".format(np.mean(seq1)))
print("Count = {0}".format(len(seq1)))
print("Mean2 = {0}".format(np.mean(seq2)))
print("Count = {0}".format(len(seq2)))

test = stats.ttest_ind(a= seq1, b= seq2, equal_var=False)  # Pop mean
print(test)

# plot histograms


bins = np.linspace(0,10,20)
plt.hist(seq1, bins, alpha=0.5)
plt.hist(seq2, bins, alpha=0.5)
plt.show()